In [15]:
import ipywidgets as widgets
from matplotlib import pyplot as plt
from ipywidgets import Image

import numpy as np
import torch

In [16]:
from ipycanvas import Canvas

canvas = Canvas(width=200, height=200)

canvas.fill_rect(25, 25, 100, 100)
canvas.clear_rect(45, 45, 60, 60)
canvas.stroke_rect(50, 50, 50, 50)

canvas

Canvas(height=200, width=200)

In [17]:
widgets.FloatSlider(value=0.5, min=0, max=1, step=0.01)

FloatSlider(value=0.5, max=1.0, step=0.01)

In [18]:
int_range = widgets.IntSlider()
output2 = widgets.Output()

display(int_range, output2)

def on_value_change(change):
    with output2:
        print(change['new'])

int_range.observe(on_value_change, names='value')

IntSlider(value=0)

Output()

In [19]:
import numpy as np

In [20]:
with open("../src/output.dat", "rb") as f:
    weights = np.load(f)

In [21]:
weights

array([-0.19275653, -0.19054382, -0.18836555, ..., -0.195931  ,
       -0.19871101, -0.19738853], dtype=float32)

In [22]:
np.max(weights)

0.015032136

In [23]:
import math

In [24]:
def connection_idx(m, n):
        # (t-1) + (t-2) + ... + (t-m) + n
        # t * m - m * (m-1) / 2
        # print(f"connection_idx({m}, {n})")
        return num_nails * m - (m + 1) * m // 2 + n - m - 1

In [25]:
with open("../src/output.dat", "rb") as f:
    weights = np.load(f)

weights = torch.nn.functional.softplus(torch.from_numpy(weights), beta=20).numpy()
print(weights.max())
print(weights.min())
print(weights.shape)

num_connections = weights.shape[0]
# quadratic formula, bitches!
num_nails = int(1/2 + np.sqrt(1/4 + 2 * num_connections))
print(num_nails)

# with open("../src/masks/mask_h=100_w=100_numnails=100_maxdist=0.1.npy", "rb") as f:
#     mask = np.load(f)

PIXELS=400

def pixel(x):
    return x * PIXELS

canvas = Canvas(width=PIXELS, height=PIXELS)

image = Image.from_file("../src/grayscale.png")
print('here')


# canvas.fill_circle(200, 200, 20)
    

canvas.stroke_style = "blue"
canvas.line_width = 0.5


min = np.min(weights)
max = np.max(weights)

def draw_strings(thresh):

    # weights = np.array([mask[y_range.value *100 + x_range.value]])
    
    for m in range(num_nails):
        for n in range(m+1, num_nails):
            conn_idx = connection_idx(m, n)
            # norm_weight = weights[0][conn_idx]
            norm_weight = (weights[conn_idx] - min)/(max-min)
            if norm_weight > thresh:
                mx = (np.cos(m / num_nails * 2 * math.pi) + 1) / 2
                my = (np.sin(m / num_nails * 2 * math.pi) + 1) / 2
                nx = (np.cos(n / num_nails * 2 * math.pi) + 1) / 2
                ny = (np.sin(n / num_nails * 2 * math.pi) + 1) / 2
                canvas.stroke_styled_line_segments(
                    np.array([[[pixel(mx), PIXELS-pixel(my)], [pixel(nx), PIXELS-pixel(ny)]]], dtype=int),
                    np.array([[0, 0, 1]], dtype=int),
                    1.0 * float(norm_weight),
                )
    
    display(canvas)

float_range = widgets.FloatSlider(value=0.5, min=0, max=1, step=0.01, continuous_update=False)
image_toggle = widgets.ToggleButtons(
    options=['Strings', 'Strings & Image', 'Image'],
    disabled=False,
    button_style='', # 'success', 'info', 'warning', 'danger' or ''
)
# x_range = widgets.IntSlider(value=50, min=0, max=100, step=1, continuous_update=False)
# y_range = widgets.IntSlider(value=50, min=0, max=100, step=1, continuous_update=False)

def draw_canvas():
    
    canvas.clear()
    canvas.stroke_circle(pixel(0.5), pixel(0.5), 200)
    if 'Image' in image_toggle.value:
        canvas.draw_image(image, 0, 0, PIXELS, PIXELS)
    if 'Strings' in image_toggle.value:
        draw_strings(float_range.value)
        

display(image_toggle, float_range)
print('here here')
draw_canvas()

def on_float_value_change(change):
    draw_canvas()

def on_toggle_value_change(change):
    draw_canvas()
    

float_range.observe(on_float_value_change, names='value')
image_toggle.observe(on_toggle_value_change, names='value')
# x_range.observe(on_toggle_value_change, names='value')
# y_range.observe(on_toggle_value_change, names='value')

0.042736225
3.4340228e-09
(32640,)
256
here


ToggleButtons(options=('Strings', 'Strings & Image', 'Image'), value='Strings')

FloatSlider(value=0.5, continuous_update=False, max=1.0, step=0.01)

here here


Canvas(height=400, width=400)